# Environment

### Imports

In [24]:
import gym
from gym import Env, spaces
import numpy as np

# Utilidad
import copy # Para copiar las variables originales a las que el ambiente edite para cuando se haga un reset

### Data Example

#### Estudiantes

In [25]:
estudiantes_json = [
    {
        "nombre": "Javier Ahumada",
        "preferencias": [
            1,
            2,
            3
        ]
    },
    {
        "nombre": "Santiago Lopez",
        "preferencias": [
            1,
            2,
            3
        ]
    },
    {
        "nombre": "Cristian Vega",
        "preferencias": [
            1,
            2,
            3
        ]
    }
]

#### Desafíos

In [26]:
desafios_json = [
    {
        "estudiantes": []
    },
    {
        "estudiantes": []
    },
    {
        "estudiantes": []
    },
    {
        "estudiantes": []
    }
]

# desafios = [[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]]

### Definición

In [ ]:
class Estudiante(object):
    def __init__(self, id, preferencias = []):
        self.id = id
        self.preferencias = preferencias # IDs de desafíos

class Desafio(object):
    def __init__(self, id, estudiantes : list = []):
        self.id = id
        self.estudiantes = estudiantes # IDs de estudiantes

    def push(self, estudiante):
        self.estudiantes.append(estudiante)

In [57]:
class OrganizationEnvironment(Env):
    def __init__(self, estudiantes_json, desafios_json):
        super(OrganizationEnvironment, self).__init__()
        self.num_estudiantes = len(estudiantes_json)
        self.num_desafios = len(desafios_json)
                
        # Espacio de observación
        self.observation_space = spaces.Dict({
            "desafios": spaces.Tuple(
                (spaces.MultiBinary(self.num_estudiantes) for _ in range(self.num_desafios))
            ),
            "estudiantes": spaces.Tuple(
                (spaces.MultiDiscrete([self.num_desafios] * 3) for _ in range(self.num_estudiantes))
            )
        })
        
        # Definición de espacio de acción
        self.action_space = spaces.MultiDiscrete([self.num_estudiantes, self.num_desafios])
        
        self.estudiantes_json = estudiantes_json
        self.desafios_json = desafios_json
        
        self.state = self.reset()
    
    def render(self):
        print(self.state)
    
    def reset(self):
        estudiantes = []
        for estudiante in self.estudiantes_json:
            estudiantes.append(estudiante["preferencias"])
        
        self.state = {
            "desafios" : [np.zeros(self.num_estudiantes, dtype=int) for _ in range(self.num_desafios)],
            "estudiantes" : estudiantes
        }
        return self.state
    
    def step(self, action):
        assert self.action_space.contains(action), "Invalid Action"
        
        if (self.check_possible(action)):
            self.state["desafios"][action[1]][action[0]] = 1
            reward = self.get_reward()
        else:
            reward = -10
        
        observation = self.state
        done = self.check_done()
        info = []
        
        return observation, reward, done, info
    
    def check_possible(self, action):
        # Verificar si estudiante ya tiene desafío asignado
        contador_de_instancias = 0
        for desafio in self.state["desafios"]:
            contador_de_instancias += desafio[action[0]]
        if(contador_de_instancias > 0):
            return False
        else:
            return True

    def get_reward(self):
        # Implementar función de reward
        return 1
    
    def check_done(self):
        # Condición para finalizar el episodio
        contador_de_estudiantes = 0
        for desafio in self.state["desafios"]:
            for estudiante in desafio:
                contador_de_estudiantes += estudiante

        if(contador_de_estudiantes == self.num_estudiantes):
            return True

        return False
        

In [59]:
env = OrganizationEnvironment(estudiantes_json, desafios_json)

while True:
    action = env.action_space.sample()
    obs, reward, done, info = env.step(action)
    
    print(reward)
    
    if done == True:
        break

print(obs)

1
1
-10
-10
1
{'desafios': [array([0, 0, 0]), array([1, 1, 0]), array([0, 0, 1]), array([0, 0, 0])], 'estudiantes': [[1, 2, 3], [1, 2, 3], [1, 2, 3]]}


In [64]:
from stable_baselines3 import A2C
from stable_baselines3.common.env_util import make_vec_env



model = A2C("CnnPolicy", env).learn(total_timesteps=10)

obs = env.reset()

while True:
    action, _states = model.predict(obs)
    obs, reward, done, info = env.step(action)
    
    print(reward)
    
    if done == True:
        break

print(obs)

/home/javo/.local/lib/python3.8/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


NotImplementedError: Nested observation spaces are not supported (Tuple/Dict space inside Tuple/Dict space).